# Preparing the environment and installing libraries:

In [2]:
!nvidia-smi

Tue Mar 11 14:43:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.04             Driver Version: 570.124.04     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A4000               Off |   00000000:01:00.0 Off |                  Off |
| 65%   81C    P2            127W /  140W |   10143MiB /  16376MiB |     70%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%pip install transformers datasets bitsandbytes peft torch sentencepiece -q
%pip install rouge-score -q
%pip install blobfile tiktoken

: 

In [7]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

import os
cache_dir = os.getenv("HF_HOME", os.path.expanduser("~/.cache/huggingface"))
print(cache_dir)
os.chdir("/")
new_cache_dir = "/Data/zakaria.abboud/.cache/huggingface"
if not os.path.exists(new_cache_dir):
    os.makedirs(new_cache_dir, exist_ok=True)

# Set the new cache directory
os.environ["HF_HOME"] = new_cache_dir
print(f"New cache directory: {new_cache_dir}")
print(f"Path from root: {os.path.abspath(new_cache_dir)}")

/Data/zakaria.abboud/.cache/huggingface
New cache directory: /Data/zakaria.abboud/.cache/huggingface
Path from root: /Data/zakaria.abboud/.cache/huggingface


In [8]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model

# Model Fine-tuning

### 1. Prepare the data

After the preparation, the dataset looks like this:
```json
{
  "title": "Document title...",
  "content": "Original document text...",
  "chosen": "Generated summary with the big model...",
  "rejected": "Generated summary with the smaller model..."
}
```
And it's split into `train`, `validation`, and `test` sets.

In [9]:
dataset = load_dataset("json", data_files={"train": "/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/data_2k/"+"train.json", "val": "/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/data_2k/"+"val.json", "test": "/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/data_2k/"+"test.json"})
# print(dataset)

In [10]:
print(dataset["train"][0]["summary"])

L'école de médecine de Salerne, fondée au XIe siècle, combinaison de traditions grecques, romaines et orientales, a joué un rôle crucial dans la médecine médiévale en considérant la médecine comme une science basée sur le raisonnement. Elle a également influencé la médecine populaire avec des ouvrages comme le "Regimen sanitatis", qui ont inspiré des guides de santé populaires jusqu'à nos jours. Cette école a développé des méthodes d'enseignement innovantes, précurseurs de celles utilisées


# 2. Custom Quantization

In [11]:

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

# Load the model in 4-bit
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


# 3. Model Definition

There are several models interesting to fine-tune for this task. We will use the `transformers` library to fine-tune a model from the list.

In [6]:
models = {
    "mT5-Base": "google/mt5-base", # 580M params
    "Qwen2.5-0.5B": "Qwen/Qwen2.5-0.5B-Instruct",
}

model_name = models["Qwen2.5-0.5B"]

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)
# print(f"Loaded model: {model_name}\n", model)
# print(model.config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
# print(f"Loaded Tokenizer: \n", tokenizer)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Now we will use the tokenizer to encode the input text.

In [22]:
def tokenize_function(examples):
    # Add EOS token manually to both content and summary
    full_prompt = f"<human>: Résume {examples['content']}\n<assistant>: {examples['summary']}"

    # Tokenize the full prompt
    tokenized_full_prompt = tokenizer(full_prompt, return_tensors="pt")
    
    # Clone the tokenized input to create labels
    labels = tokenized_full_prompt["input_ids"].clone()

    prompt = full_prompt[:full_prompt.find("<assistant>")] + "<assistant>:"

    end_prompt_index = tokenizer(prompt, return_tensors="pt").input_ids.shape[1]

    labels[:, :end_prompt_index] = -100

    return {
        'input_ids': tokenized_full_prompt["input_ids"].flatten(),
        'labels': labels.flatten(),
        'attention_mask': tokenized_full_prompt["attention_mask"].flatten(),
    }

# Tokenize the dataset

data = dataset["train"].shuffle(seed=42).map(tokenize_function)
    

# data = dataset["train"].shuffle(seed=42).map(tokenize_function, batched=True)

Map:   0%|          | 0/4548 [00:00<?, ? examples/s]

We will use Lora to fine-tune the model efficiently without having to adjust all the parameters.

In [27]:
# Extract target_modules
target_modules = [
    'gate_proj',
    'up_proj',
    'down_proj',
]

config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)


model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 13,271,040 || all params: 507,303,808 || trainable%: 2.6160


In [29]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.do_sample = True

In [28]:
%%time
article = """Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick.\n\n\n== Histoire ==\nLe Monument-Lefebvre est inauguré les 16 et 17 juin 1897 et nommé ainsi en l'honneur du père Camille Lefebvre, fondateur du Collège Saint-Joseph, décédé deux ans auparavant.\nIl est tout d'abord utilisé comme laboratoire par le Collège Saint-Joseph, tandis que son amphithéâtre permet d'organiser des concerts, des conférences et de nombreuses manifestations diverses. Le lieu et son fondateur ont joué un rôle déterminant dans la renaissance acadienne.\nAprès la fermeture du Collège en 1972, le Monument-Lefebvre manque de disparaître mais est finalement rénové et déclaré lieu historique national en 1994.\n\n\n== Architecture ==\nLe Monument-Lefebvre est construit en grès rustiqué de couleur olive. Sa façade symétrique comprend des éléments néoromans. Il abrite un théâtre et des salle de classe.\n\n\n== Notes et références ==\n\n\n== Liens externes ==\nRessources relatives à l'architecture : Édifices fédéraux patrimoniaux Lieux historiques nationaux Répertoire canadien des lieux patrimoniaux \n(fr + en) Site officiel\n\n Portail de l’Acadie   Portail du Nouveau-Brunswick   Portail des lieux patrimoniaux du Canada"""

prompt = f"""
<human>: résume {article}
<assistant>:
"""

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

NameError: name 'generation_config' is not defined

In [33]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=3,
    logging_steps=20,
    output_dir="/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/results",
    max_steps=500,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    label_names=["labels"],
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
trainer = Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)

model.config.use_cache = False
trainer.train()

Step,Training Loss
20,0.909900
40,0.919300
60,0.973800
80,1.006500
100,1.020900
120,1.005200
140,1.002600
160,0.804800
180,0.769600
200,0.778600


TrainOutput(global_step=500, training_loss=0.7453178424835205, metrics={'train_runtime': 3151.5771, 'train_samples_per_second': 5.077, 'train_steps_per_second': 0.159, 'total_flos': 4.016179539972557e+16, 'train_loss': 0.7453178424835205, 'epoch': 3.499560246262093})

# 4. Test the fine-tuned model

In [35]:
%%time

article = """Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick.\n\n\n== Histoire ==\nLe Monument-Lefebvre est inauguré les 16 et 17 juin 1897 et nommé ainsi en l'honneur du père Camille Lefebvre, fondateur du Collège Saint-Joseph, décédé deux ans auparavant.\nIl est tout d'abord utilisé comme laboratoire par le Collège Saint-Joseph, tandis que son amphithéâtre permet d'organiser des concerts, des conférences et de nombreuses manifestations diverses. Le lieu et son fondateur ont joué un rôle déterminant dans la renaissance acadienne.\nAprès la fermeture du Collège en 1972, le Monument-Lefebvre manque de disparaître mais est finalement rénové et déclaré lieu historique national en 1994.\n\n\n== Architecture ==\nLe Monument-Lefebvre est construit en grès rustiqué de couleur olive. Sa façade symétrique comprend des éléments néoromans. Il abrite un théâtre et des salle de classe.\n\n\n== Notes et références ==\n\n\n== Liens externes ==\nRessources relatives à l'architecture : Édifices fédéraux patrimoniaux Lieux historiques nationaux Répertoire canadien des lieux patrimoniaux \n(fr + en) Site officiel\n\n Portail de l’Acadie   Portail du Nouveau-Brunswick   Portail des lieux patrimoniaux du Canada"""

prompt = f"""
<human>: Résume {article}
<assistant>:
"""

encoding = tokenizer(prompt, padding="max_length", truncation=True, max_length=2048, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<human>: Résume Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick.


== Histoire ==
Le Monument-Lefebvre est inauguré les 16 et 17 juin 1897 et nommé ainsi en l'honneur du père Camille Lefebvre, fondateur du Collège Saint-Joseph, décédé deux ans auparavant.
Il est tout d'abord utilisé comme laboratoire par le Collège Saint-Joseph, tandis que son amphithéâtre permet d'organiser des concerts, des conférences et de nombreuses manifestations diverses. Le lieu et son fondateur ont joué un rôle déterminant dans la renaissance acadienne.
Après la fermeture du Collège en 1972, le Monument-Lefebvre manque de disparaître mais est finalement rénové et déclaré lieu historique national en 1994.


== Architecture ==
Le Monument-Lefebvre est construit en grès rustiqué de couleur olive. Sa façade symétrique comprend des éléments néoromans. Il abrite un théâtre et des salle de classe.


== Notes et références ==


== Liens externes ==
Ressources re

# 3. Save the fine-tuned model

In [39]:
# save the model
finetunedmodel = model.merge_and_unload()
finetunedmodel.save_pretrained("/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/models/finetuned_2k_token")
finetunedmodel
# load the model
# model_2 = AutoModelForCausalLM.from_pretrained("/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/models")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
              (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
              (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
              (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
            )
            (mlp): Qwen2MLP(
              (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
              (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
              (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
              (act_fn): SiLU()
            )
            (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
 

### Generating the new summaries with the fine-tuned model

In [40]:
def summarize_document(document):
    # Tokenize the input document
    prompt = f"<human>: Summarize the following text :\n\n{document}\n\nYou should respond in French, with 5 sentences maximum.\n\n<assistant>:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate summary
    summary_ids = model.generate(**inputs, max_new_tokens=128)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    assistant_index = summary.find("<assistant>:")
    if assistant_index != -1:
        summary = summary[assistant_index + len("<assistant>:"):].strip()

    return summary

In [42]:
import json
# test
articles = json.load(open('/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/summaries/2k_wikipedia_articles.json', 'r', encoding='utf-8'))


(6498, 460)

In [43]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm


# File to save the summarized articles
output_file = '/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/summaries/2k_finetuned_summarized_wikipedia_articles.json'

# Function to save results progressively
def save_progress(summarized_articles, output_file):
    if os.path.exists(output_file):
        with open(output_file, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
    else:
        existing_data = []

    existing_data.extend(summarized_articles)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

# Summarize articles in parallel
summarized_articles = []
with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your system
    futures = {executor.submit(summarize_document, article['content']): article for article in articles}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Summarizing articles"):
        result = future.result()
        summarized_articles.append({
            'title': futures[future]['title'],
            'content': futures[future]['content'],
            'summary': result
        })

        # Save progress every 10 summaries
        if len(summarized_articles) % 10 == 0:
            save_progress(summarized_articles, output_file)
            summarized_articles = []  # Clear the list after saving

# Save any remaining summaries
if summarized_articles:
    save_progress(summarized_articles, output_file)

print(f"Summarized articles saved to '{output_file}'")

Summarizing articles: 100%|██████████| 6498/6498 [4:46:55<00:00,  2.65s/it]  


Summarized articles saved to '/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/summaries/2k_finetuned_summarized_wikipedia_articles.json'


# 5. Evaluate the model

### 5.1 ROUGE Score

In [ ]:
from evaluate import load

rouge = load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return result

results = compute_metrics(trainer.predict(tokenized_datasets["test"]))

print(results)

: 

### 5.2 Bert Score

: 